In [1]:
import torch
import numpy as np
from pathlib import Path
import plotly.graph_objs as go
import quaternion


from initialization import TargetImageDataset

In [2]:
target_image_dataset = TargetImageDataset(
    Path("../data/calculator/raw_images"),
    Path("../data/image_pose.json")
)

fg_V_sdf = torch.load("../data/initial_v_sdf_fg.pt", weights_only=True)
bg_V_sdf = torch.load("../data/initial_v_sdf_bg.pt", weights_only=True)

print(fg_V_sdf)
print(bg_V_sdf)
print(len(target_image_dataset))

tensor([[[[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 0.9900,  ..., 1.0000, 1.0000, 1.0000],
           ...,
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]],

          [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           ...,
           [1.0000, 1.0000, 0.9900,  ..., 0.9900, 1.0000, 1.0000],
           [0.9900, 1.0000, 0.9900,  ..., 1.0000, 1.0000, 1.0000],
           [0.9900, 1.0000, 0.9900,  ..., 1.0000, 1.0000, 1.0000]],

          [[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
           [1.0000, 1.0000

In [3]:
data, pose = target_image_dataset[0]

# q = np.quaternion(*pose.qvec)

# print(q.w, q.x, q.y, q.z)

# quaternion.as_spherical_coords(q)

qm = quaternion.as_rotation_matrix(pose.qvec)   # 3x3 rot mat
# print(qm)

tx, ty, tz = pose.tvec 

In [4]:
# https://github.com/colmap/colmap/blob/main/src/colmap/sensor/models.h#L715
# https://calib.io/blogs/knowledge-base/camera-models?srsltid=AfmBOoosTUXUe3QZqSrWoJXC9Yr04axC6Mvx7ru4xjo-yHMRf4H_erhx

GRAPH_RESOLUTION = 256.0

px, py = np.mgrid[-pose.width/2:pose.width/2:GRAPH_RESOLUTION,
                      -pose.height/2:pose.height/2:GRAPH_RESOLUTION]+0.5
pz = np.full(px.shape, 1.0)

# ox, oy, oz = np.full(px.shape, 0.0), np.full(px.shape, 0.0), np.full(px.shape, 1.0)

#### ASPECT RATIO (f)
img_points = np.dstack([px.flatten()/pose.f, py.flatten()/pose.f, pz.flatten()])[0]  # Nx2 points to be rot'd at origin
# img_offset = np.array([0.0, 0.0, 1.0])  # normalized points offset w=1 (z=1)

##### DISTORTION (k)
distortion = pose.k*(img_points[:,0]**2+img_points[:,1]**2)

dx = img_points[:,0]*(1+distortion)
dy = img_points[:,1]*(1+distortion)
dz = img_points[:,2]
distorted_img_points = np.dstack([dx, dy, dz])[0]

##### ROTATION (qvec)

# ji,ni->ni

rotated_img_points = np.einsum("ij,nj->ni", qm, distorted_img_points)
# rotated_img_offset = np.einsum("ij,j->i", qm, img_offset)

rpx, rpy, rpz = np.split(rotated_img_points, 3, axis=1)
# rox, roy, roz = rotated_img_offset
# rpx, rpy, rpz = rpx.flatten()+rox, rpy.flatten()+roy, rpz.flatten()+roz
rpx, rpy, rpz = rpx.flatten(), rpy.flatten(), rpz.flatten()

In [5]:
go.Figure(data=[
    go.Scatter3d(
        x=[0],
        y=[0],
        z=[0],
        mode='markers',
        marker={
            'size': 10,
            'opacity': 1.0,
        }
    ),
    go.Scatter3d(
        x=rpx,
        y=rpy,
        z=rpz,
        mode='markers',
        marker={
            'size': 5,
            'opacity': 1.0,
        }
    ),
], layout=go.Layout(
    margin={'l': 0, 'r': 0, 'b': 0, 't': 0},
    scene=dict(
        aspectmode="cube",
        xaxis=dict(range=[-1.1, 1.1]),  # Set x-axis range
        yaxis=dict(range=[-1.1, 1.1]),  # Set y-axis range
        zaxis=dict(range=[-1.1, 1.1])   # Set z-axis range
    )
))